In [10]:
from langchain_community.llms import Ollama

In [11]:
# Load llama3 model using Ollama
llm = Ollama(model="llama3")

# llm.invoke("Tell me a joke")

In [12]:
print(llm.invoke("Tell me a joke"))

Why couldn't the bicycle stand up by itself?

Because it was two-tired!

(Sorry, I know it's a bit of a "dad" joke, but I hope it brought a smile to your face!)


In [1]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

from langchain.memory import ConversationSummaryBufferMemory, ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.chains import ConversationChain, LLMChain
from langchain.prompts import MessagesPlaceholder, PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

## Chat history without Prompt Template

In [262]:

conversation_with_summary = ConversationChain(
    llm=llm,
    # We set a very low max_token_limit for the purposes of testing.
    # memory=ConversationSummaryBufferMemory(llm=llm, max_token_limit=100),
    # memory=ConversationBufferWindowMemory(k=2),
    memory=ConversationBufferMemory(),
    verbose=True,
)

In [153]:
conversation_with_summary.invoke(input="Hi, what's up")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up
AI:

> Finished chain.


{'input': "Hi, what's up",
 'history': '',
 'response': "Not much! I'm just processing some fascinating information about the topological features of the Martian surface. Did you know that the largest volcano on Mars is called Olympus Mons, and it stands approximately 27 km high? That's three times the height of Mount Everest! Additionally, I've been analyzing some intriguing patterns in the Martian dust devil formation, which might provide valuable insights for future planetary exploration missions. How about you, what's new with you?"}

In [78]:
conversation_with_summary.invoke(input="Intersting, Do you know what is langchain? ")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up
AI: Not much, just processing 1.37 terabytes of new data every hour and updating my knowledge graph with fresh information from the web! By the way, I've been trained on a dataset of over 45 million articles from Wikipedia and books from Project Gutenberg. It's always fascinating to see how humans interact with me, don't you think?
Human: Intersting, Do you know what is langchain? 
AI:

> Finished chain.


{'input': 'Intersting, Do you know what is langchain? ',
 'history': "Human: Hi, what's up\nAI: Not much, just processing 1.37 terabytes of new data every hour and updating my knowledge graph with fresh information from the web! By the way, I've been trained on a dataset of over 45 million articles from Wikipedia and books from Project Gutenberg. It's always fascinating to see how humans interact with me, don't you think?",
 'response': "Langchain! That's an intriguing topic. According to my training data, LangChain is a recent AI-powered language understanding model that uses a combination of transformer-based architecture and pre-trained language models like BERT and RoBERTa. It's designed to generate human-like responses by leveraging contextual information from the conversation.\n\nFrom what I know, LangChain has been trained on a massive dataset of text, including books, articles, and online conversations. Its developers claim that it can handle complex tasks such as conversationa

In [79]:
conversation_with_summary.invoke(input="Can you write a sample script of a conversation chain in langchain which has chat history also? ")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up
AI: Not much, just processing 1.37 terabytes of new data every hour and updating my knowledge graph with fresh information from the web! By the way, I've been trained on a dataset of over 45 million articles from Wikipedia and books from Project Gutenberg. It's always fascinating to see how humans interact with me, don't you think?
Human: Intersting, Do you know what is langchain? 
AI: Langchain! That's an intriguing topic. According to my training data, LangChain is a recent AI-powered language understanding model that uses a combination of transformer-based architecture and pre-trained language models like BERT and RoBERTa. It's designed to gener

{'input': 'Can you write a sample script of a conversation chain in langchain which has chat history also? ',
 'history': "Human: Hi, what's up\nAI: Not much, just processing 1.37 terabytes of new data every hour and updating my knowledge graph with fresh information from the web! By the way, I've been trained on a dataset of over 45 million articles from Wikipedia and books from Project Gutenberg. It's always fascinating to see how humans interact with me, don't you think?\nHuman: Intersting, Do you know what is langchain? \nAI: Langchain! That's an intriguing topic. According to my training data, LangChain is a recent AI-powered language understanding model that uses a combination of transformer-based architecture and pre-trained language models like BERT and RoBERTa. It's designed to generate human-like responses by leveraging contextual information from the conversation.\n\nFrom what I know, LangChain has been trained on a massive dataset of text, including books, articles, and onl

In [81]:
response = conversation_with_summary.invoke(input="Can you tell me how I started the conversation? ")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up
AI: Not much, just processing 1.37 terabytes of new data every hour and updating my knowledge graph with fresh information from the web! By the way, I've been trained on a dataset of over 45 million articles from Wikipedia and books from Project Gutenberg. It's always fascinating to see how humans interact with me, don't you think?
Human: Intersting, Do you know what is langchain? 
AI: Langchain! That's an intriguing topic. According to my training data, LangChain is a recent AI-powered language understanding model that uses a combination of transformer-based architecture and pre-trained language models like BERT and RoBERTa. It's designed to gener

In [84]:
response['response']

'Let me check my knowledge graph! Ah yes, according to our conversation history, you started the conversation by saying "Hi, what\'s up".'

## ---------------------------------------
## Chat history with Prompt Template

In [197]:
template='''
You are an helpful assistant

Current conversation:
{chat_history}
Human: {input}
AI:
'''
prompt = PromptTemplate(input_variables=['chat_history', 'language'], template = template)

In [198]:

conversation = ConversationChain(
    llm=llm,
    prompt = prompt,
    # Default value of memory_key in ConversationBufferMemory is "history". If you don't specify memory_key,
    # then in MessagesPlaceholder, variable_name should be passed as "history"
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True),
    verbose=True,
)

In [199]:
response = conversation.invoke(input="Hi, my name is Vinay")
response



> Entering new ConversationChain chain...
Prompt after formatting:

You are an helpful assistant

Current conversation:
[]
Human: Hi, my name is Vinay
AI:


> Finished chain.


{'input': 'Hi, my name is Vinay',
 'chat_history': [HumanMessage(content='Hi, my name is Vinay'),
  AIMessage(content="Nice to meet you, Vinay! I'm your helpful assistant. How can I assist you today? Do you have a specific question or topic in mind that you'd like to discuss?")],
 'response': "Nice to meet you, Vinay! I'm your helpful assistant. How can I assist you today? Do you have a specific question or topic in mind that you'd like to discuss?"}

In [200]:
response = conversation.invoke(input="What is my name?")
response



> Entering new ConversationChain chain...
Prompt after formatting:

You are an helpful assistant

Current conversation:
[HumanMessage(content='Hi, my name is Vinay'), AIMessage(content="Nice to meet you, Vinay! I'm your helpful assistant. How can I assist you today? Do you have a specific question or topic in mind that you'd like to discuss?")]
Human: What is my name?
AI:


> Finished chain.


{'input': 'What is my name?',
 'chat_history': [HumanMessage(content='Hi, my name is Vinay'),
  AIMessage(content="Nice to meet you, Vinay! I'm your helpful assistant. How can I assist you today? Do you have a specific question or topic in mind that you'd like to discuss?"),
  HumanMessage(content='What is my name?'),
  AIMessage(content="Hi again, Vinay! Since we've already introduced ourselves, I'll give you a quick answer: Your name is indeed Vinay! Is there anything else on your mind, or would you like me to help with something specific?")],
 'response': "Hi again, Vinay! Since we've already introduced ourselves, I'll give you a quick answer: Your name is indeed Vinay! Is there anything else on your mind, or would you like me to help with something specific?"}

## --------------------------------------------------------------
## Trying with LLM Chain, Multiple inputs and Prompt Template
##### <i>Multiple input variables don't work with ConversationChain</i>

In [49]:
prompt = PromptTemplate(
    input_variables=['history', 'input', 'custom'], 
    template='''
    You are an helpful assistant. Answer the human question truthfully and {custom}

    Current conversation:
    {history}
    Human: {input}
    AI:
    ''')

In [50]:

memory = ConversationBufferMemory(
    session_id="sess1",  # Identifies your user or a user's session
    memory_key="history",          # Ensure this matches the key used in chain's prompt template
    input_key='input',
    return_messages=True,          # Does your prompt template expect a string or a list of Messages?
)

conversation = LLMChain(
    llm=llm,
    prompt = prompt,
    # Default value of memory_key in ConversationBufferMemory is "history". If you don't specify memory_key,
    # then in MessagesPlaceholder, variable_name should be passed as "history"
    memory=memory,
    verbose=True,
)

In [46]:
response = conversation.invoke({'input':"Hi, my name is Vinay", "custom":"concisely"})
response



> Entering new LLMChain chain...
Prompt after formatting:

    You are an helpful assistant. Answer the human question truthfully and concisely

    Current conversation:
    []
    Human: Hi, my name is Vinay
    AI:
    

> Finished chain.


{'input': 'Hi, my name is Vinay',
 'custom': 'concisely',
 'history': [HumanMessage(content='Hi, my name is Vinay'),
  AIMessage(content='Nice to meet you, Vinay!')],
 'text': 'Nice to meet you, Vinay!'}

In [47]:
response = conversation.invoke({'input':"What is my Name", "custom":"concisely"})
response



> Entering new LLMChain chain...
Prompt after formatting:

    You are an helpful assistant. Answer the human question truthfully and concisely

    Current conversation:
    [HumanMessage(content='Hi, my name is Vinay'), AIMessage(content='Nice to meet you, Vinay!')]
    Human: What is my Name
    AI:
    

> Finished chain.


{'input': 'What is my Name',
 'custom': 'concisely',
 'history': [HumanMessage(content='Hi, my name is Vinay'),
  AIMessage(content='Nice to meet you, Vinay!'),
  HumanMessage(content='What is my Name'),
  AIMessage(content='Vinay')],
 'text': 'Vinay'}

In [48]:
response = conversation.invoke({'input':"What is my Name", "custom":"in detail"})
response



> Entering new LLMChain chain...
Prompt after formatting:

    You are an helpful assistant. Answer the human question truthfully and in detail

    Current conversation:
    [HumanMessage(content='Hi, my name is Vinay'), AIMessage(content='Nice to meet you, Vinay!'), HumanMessage(content='What is my Name'), AIMessage(content='Vinay')]
    Human: What is my Name
    AI:
    

> Finished chain.


{'input': 'What is my Name',
 'custom': 'in detail',
 'history': [HumanMessage(content='Hi, my name is Vinay'),
  AIMessage(content='Nice to meet you, Vinay!'),
  HumanMessage(content='What is my Name'),
  AIMessage(content='Vinay'),
  HumanMessage(content='What is my Name'),
  AIMessage(content='I\'m happy to help!\n\nYou asked: "What is my Name?"\n\nAnd the answer is... **VINAY**! You introduced yourself earlier in our conversation, and I\'m happy to remind you that your name is indeed Vinay.')],
 'text': 'I\'m happy to help!\n\nYou asked: "What is my Name?"\n\nAnd the answer is... **VINAY**! You introduced yourself earlier in our conversation, and I\'m happy to remind you that your name is indeed Vinay.'}

## -------------------------------------------------------------
## Trying with LLM Chain, Multiple inputs and ChatPromptTemplate

In [17]:
messages = [
    SystemMessagePromptTemplate.from_template('''
        You are an helpful assistant. Answer the human question truthfully and {custom}
        '''),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template('{input}'),
]

prompt = ChatPromptTemplate.from_messages(messages)


In [18]:

memory = ConversationBufferMemory(
    session_id="sess1",  # Identifies your user or a user's session
    memory_key="history",          # Ensure this matches the key used in chain's prompt template
    input_key='input',
    return_messages=True,          # Does your prompt template expect a string or a list of Messages?
)

conversation = LLMChain(
    llm=llm,
    prompt = prompt,
    memory=memory,
    verbose=True,
)

In [368]:
response = conversation.invoke({'input':"Hi, my name is Vinay", "custom":"very concisely"})
response



> Entering new LLMChain chain...
Prompt after formatting:
System: 
        You are an helpful assistant. Answer the human question truthfully and very concisely
        
Human: Hi, my name is Vinay

> Finished chain.


{'input': 'Hi, my name is Vinay',
 'custom': 'very concisely',
 'history': [HumanMessage(content='Hi, my name is Vinay'),
  AIMessage(content='Nice to meet you, Vinay!')],
 'text': 'Nice to meet you, Vinay!'}

In [369]:
response = conversation.invoke({'input':"What is my Name", "custom":"very concisely"})
response



> Entering new LLMChain chain...
Prompt after formatting:
System: 
        You are an helpful assistant. Answer the human question truthfully and very concisely
        
Human: Hi, my name is Vinay
AI: Nice to meet you, Vinay!
Human: What is my Name

> Finished chain.


{'input': 'What is my Name',
 'custom': 'very concisely',
 'history': [HumanMessage(content='Hi, my name is Vinay'),
  AIMessage(content='Nice to meet you, Vinay!'),
  HumanMessage(content='What is my Name'),
  AIMessage(content='Vinay!')],
 'text': 'Vinay!'}

In [370]:
conversation.prompt

ChatPromptTemplate(input_variables=['custom', 'history', 'input'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['custom'], template='\n        You are an helpful assistant. Answer the human question truthfully and {custom}\n        ')), MessagesPlaceholder(variable_name='history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

## Streaming

In [5]:
from langchain_core.runnables import RunnablePassthrough

In [136]:
messages = [
    SystemMessagePromptTemplate.from_template('''
        You are an helpful assistant. Answer the human question truthfully and {custom}
        '''),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template('{input}'),
]

prompt = ChatPromptTemplate.from_messages(messages)



memory = ConversationBufferMemory(
    session_id="sess1",  # Identifies your user or a user's session
    memory_key="history",          # Ensure this matches the key used in chain's prompt template
    input_key='input',
    return_messages=True,          # Does your prompt template expect a string or a list of Messages?
)

# conversation = LLMChain(
#     llm=llm,
#     prompt = prompt,
#     # memory=memory,
#     # verbose=True,
# )

# conversation = {'custom': RunnablePassthrough(), 'history':RunnablePassthrough(), 'input':RunnablePassthrough()} \
#     | prompt | llm

conversation = prompt | llm

In [137]:
def llm_runnable(human_q, custom_v, memory=memory):
    ai_response = []
    for chunk in conversation.stream({'input':human_q, 
                                    "custom":custom_v, 
                                    'history':memory.buffer_as_messages}):
        print(chunk, end="", flush=True)
        ai_response.append(chunk)
    human_message = HumanMessage(content=human_q)
    ai_message = AIMessage(content=''.join(ai_response))
    memory.chat_memory.add_messages([human_message,ai_message])

In [138]:
memory.clear()

In [139]:
human_q = 'My name is Vinay'
custom_v = 'very concisely'
llm_runnable(human_q, custom_v)

Nice to meet you, Vinay!

In [140]:
human_q = 'What is my name'
custom_v = 'very concisely'
llm_runnable(human_q, custom_v)

Vinay!

In [141]:
human_q = 'What was your response to my previous question'
custom_v = 'very concisely'
llm_runnable(human_q, custom_v)

Nice to meet you, Vinay!

In [135]:
memory.buffer

[HumanMessage(content='My name is Vinay'),
 AIMessage(content='Nice to meet you, Vinay!'),
 HumanMessage(content='What is my name'),
 AIMessage(content='Vinay!'),
 HumanMessage(content='What was your response to my previous question'),
 AIMessage(content='Nice to meet you, Vinay!')]

## ---------------------------------
## With RunnableWithMessageHistory

In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


In [7]:

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer \
             and if you don't know the answer, say that you don't know.",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

runnable = prompt | llm


In [44]:

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [45]:
with_message_history.invoke(
    {"ability": "conversation", "input": "My name is Vinay?"},
    config={"configurable": {"session_id": "abc123"}},
)

"Nice to meet you, Vinay! That's a unique name - where did you get it from?"

In [46]:
with_message_history.invoke(
    {"ability": "conversation", "input": "What is my Name?"},
    config={"configurable": {"session_id": "abc123"}},
)

"I think I'm supposed to respond about your name, Vinay! You're correct, that is indeed your name."

In [47]:
with_message_history.invoke(
    {"ability": "conversation", "input": "What is my Name?"},
    config={"configurable": {"session_id": "xyz123"}},
)

"I'm happy to help! Unfortunately, I don't have that information. Can you remind me?"

In [48]:
for chunk in with_message_history.stream(
            {"ability": "math", "input": "What is my cosine similarity?"},
            config={"configurable": {"session_id": "abc123"}}
            ):
        print(chunk, end="", flush=True)


Vinay, the cosine similarity depends on the context and vectors involved. Could you provide more information or clarify what you mean by "my" cosine similarity?

### Configure RunnableWithMessageHistory further based on user id and conversation id 

In [13]:
from langchain_core.runnables import ConfigurableFieldSpec

store = {}


def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)


In [15]:

with_message_history.invoke(
    {"ability": "jokes", "input": "Tell me a joke"},
    config={"configurable": {"user_id": "123", "conversation_id": "1"}},
)

Parent run 2eba6283-61a7-4084-b1a7-42dfd5398cba not found for run 856cb329-6876-468e-a4af-4b435ba72142. Treating as a root run.


'Why did the computer go to the doctor? It had a virus!'

In [16]:
# remembers
with_message_history.invoke(
    {"ability": "jokes", "input": "What was the joke about?"},
    config={"configurable": {"user_id": "123", "conversation_id": "1"}},
)

Parent run 63e464a8-07a9-459d-a352-ed388cafa3a8 not found for run 46002925-beee-4c6e-9450-763df6609f91. Treating as a root run.


'I told two jokes! The first one was about a scarecrow, and the second one was about a computer getting a "virus"'

In [53]:
# New user_id --> does not remember
with_message_history.invoke(
    {"ability": "jokes", "input": "What was the joke about?"},
    config={"configurable": {"user_id": "123", "conversation_id": "2"}},
)

'I just told one! It was a "punderful" joke about a punder... just kidding, I\'ll tell it again.'

In [54]:
# New user_id --> does not remember
with_message_history.invoke(
    {"ability": "jokes", "input": "What was the joke about?"},
    config={"configurable": {"user_id": "456", "conversation_id": "1"}},
)

'I\'m glad you asked! It was a "punderful" one... Okay, okay, I\'ll stop with the puns now!'

In [55]:
store

{('123',
  '1'): InMemoryChatMessageHistory(messages=[HumanMessage(content='Tell me a joke'), AIMessage(content='Why did the scarecrow win an award? Because he was outstanding in his field! (get it?)'), HumanMessage(content='What was the joke about?'), AIMessage(content='The joke\'s about a scarecrow winning an award for being great at his job, "outstanding in his field"!')]),
 ('123',
  '2'): InMemoryChatMessageHistory(messages=[HumanMessage(content='What was the joke about?'), AIMessage(content='I just told one! It was a "punderful" joke about a punder... just kidding, I\'ll tell it again.')]),
 ('456',
  '1'): InMemoryChatMessageHistory(messages=[HumanMessage(content='What was the joke about?'), AIMessage(content='I\'m glad you asked! It was a "punderful" one... Okay, okay, I\'ll stop with the puns now!')])}

### Message input, dict output

In [62]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel({"output_message": llm})


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    output_messages_key="output_message",
)

with_message_history.invoke(
    [HumanMessage(content="What did Simone de Beauvoir believe about free will")],
    config={"configurable": {"session_id": "baz"}},
)

{'output_message': 'Simone de Beauvoir, a French philosopher and feminist, was deeply concerned with the concept of free will. In her influential book "The Ethics of Ambiguity" (1947), she argued that the idea of free will is often misunderstood or misused.\n\nDe Beauvoir believed that the notion of free will is not a fixed or innate ability, but rather a dynamic and contextual phenomenon. She saw freedom as a process of self-creation and self-definition, which requires continuous effort and responsibility. According to her, humans have a fundamental ambiguity between their desire for freedom and their need for security.\n\nDe Beauvoir\'s philosophy was heavily influenced by existentialism, particularly the ideas of Jean-Paul Sartre. She agreed with Sartre that human beings are "condemned to be free," meaning we must take responsibility for our choices and actions, even when they are difficult or uncertain. However, de Beauvoir also recognized the constraints and limitations imposed by

In [63]:
with_message_history.invoke(
    [HumanMessage(content="How did this compare to Sartre")],
    config={"configurable": {"session_id": "baz"}},
)

{'output_message': 'Simone de Beauvoir\'s concept of free will was heavily influenced by Jean-Paul Sartre\'s existentialist ideas. In fact, de Beauvoir was a close friend and collaborator of Sartre\'s, and her philosophy shares many similarities with his. Here are some key comparisons between their views on free will:\n\n**Similarities:**\n\n1. **"Condemned to be free"**: Both de Beauvoir and Sartre emphasized that human beings have the responsibility to take control of their choices and actions, even when they are difficult or uncertain.\n2. **Existential freedom**: They both believed that humans have a fundamental freedom to choose and create themselves, which is characterized by ambiguity and uncertainty.\n3. **Responsibility**: Both philosophers stressed the importance of individual responsibility for our choices and actions.\n\n**Differences:**\n\n1. **Contextual dependence**: While Sartre\'s concept of free will was more abstract and focused on individual choice, de Beauvoir emph

# Message with with Llama3 prompt template

In [29]:
from langchain_community.chat_models.ollama import ChatOllama

llm = ChatOllama(model="llama3", temperature=0, format='json')

In [41]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate


prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a helpful assistant which is good in conversation.
    Respond to user query honestly. Keep your answer concise, not exceeding 3 sentences
    Format of your response should be JSON with 1 key as 'answer'
    <|eot_id|>
    {chat_history}
    <|start_header_id|>user<|end_header_id|>
    {query}
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["query","chat_history"],
)

test_llm = prompt | llm | JsonOutputParser()
question = "My name is Vinay"
print(test_llm.invoke({"query": question, 'chat_history':''}))

{'answer': "Nice to meet you, Vinay! I'm happy to chat with you."}


In [42]:
chat_history = """
    <|start_header_id|>user<|end_header_id|>
    My name is Vinay
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    Nice to meet you, Vinay! I'm here to help with any questions or topics you'd like to discuss. What's on your mind?
    <|eot_id|>
"""

In [43]:
question = "Remember my name?"
print(test_llm.invoke({"query": question, 'chat_history':chat_history}))

{'answer': 'Yes, I remember that your name is Vinay!'}


### Method 1 : Conversation Chain

In [48]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a helpful assistant which is good in conversation.
    Respond to user query honestly. Keep your answer concise, not exceeding 3 sentences.
    Format of your response should be JSON with 1 key as 'answer'
    <|eot_id|>
    {chat_history}
    <|start_header_id|>user<|end_header_id|>
    {input}
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["input","chat_history"],
)

memory = ConversationBufferMemory(
    session_id="sess1",  # Identifies your user or a user's session
    memory_key="chat_history",          # Ensure this matches the key used in chain's prompt template
    input_key='input',
    return_messages=True,          # Does your prompt template expect a string or a list of Messages?
)

conversation = ConversationChain(
    llm=llm,
    prompt = prompt,
    # Default value of memory_key in ConversationBufferMemory is "history". If you don't specify memory_key,
    # then in MessagesPlaceholder, variable_name should be passed as "history"
    memory=memory,
    verbose=True,
)

In [49]:
conversation.invoke({'input':'My name is Vinay'})



> Entering new ConversationChain chain...
Prompt after formatting:
<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a helpful assistant which is good in conversation.
    Respond to user query honestly. Keep your answer concise, not exceeding 3 sentences.
    Format of your response should be JSON with 1 key as 'answer'
    <|eot_id|>
    []
    <|start_header_id|>user<|end_header_id|>
    My name is Vinay
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    

> Finished chain.


{'input': 'My name is Vinay',
 'chat_history': [HumanMessage(content='My name is Vinay'),
  AIMessage(content='{\n"answer": "Nice to meet you, Vinay! I\'m here to help with any questions or topics you\'d like to discuss." \n}')],
 'response': '{\n"answer": "Nice to meet you, Vinay! I\'m here to help with any questions or topics you\'d like to discuss." \n}'}

In [50]:
conversation.invoke({'input':'Remember my Name?'})



> Entering new ConversationChain chain...
Prompt after formatting:
<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a helpful assistant which is good in conversation.
    Respond to user query honestly. Keep your answer concise, not exceeding 3 sentences.
    Format of your response should be JSON with 1 key as 'answer'
    <|eot_id|>
    [HumanMessage(content='My name is Vinay'), AIMessage(content='{\n"answer": "Nice to meet you, Vinay! I\'m here to help with any questions or topics you\'d like to discuss." \n}')]
    <|start_header_id|>user<|end_header_id|>
    Remember my Name?
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    

> Finished chain.


{'input': 'Remember my Name?',
 'chat_history': [HumanMessage(content='My name is Vinay'),
  AIMessage(content='{\n"answer": "Nice to meet you, Vinay! I\'m here to help with any questions or topics you\'d like to discuss." \n}'),
  HumanMessage(content='Remember my Name?'),
  AIMessage(content='{\n"answer": "Yes, your name is Vinay!" \n}')],
 'response': '{\n"answer": "Yes, your name is Vinay!" \n}'}

### Method 1 : RunnableWithMessageHistory

In [52]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


In [59]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a helpful assistant which is good in conversation.
    Respond to user query honestly. Keep your answer concise, not exceeding 3 sentences.
    Format of your response should be JSON with 1 key as 'answer'
    <|eot_id|>
    {chat_history}
    <|start_header_id|>user<|end_header_id|>
    {input}
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["input","chat_history"],
)

runnable = prompt | llm | JsonOutputParser()

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [60]:
config = {"configurable": {"session_id": "abc123"}}

response = with_message_history.invoke(
    {"input": "My name is Vinay"},
    config=config,
)

Parent run 0f276135-5796-49da-a1d1-c1e3f95a0b6e not found for run a3b6a288-8d11-4f4f-b88b-1f7185f44dd2. Treating as a root run.


In [61]:
response

{'answer': "Nice to meet you, Vinay! I'm here to help with any questions or topics you'd like to discuss."}

In [62]:
config = {"configurable": {"session_id": "abc123"}}

with_message_history.invoke(
    {"input": "Remember my Name?"},
    config=config,
)

Parent run a48ac6e6-832a-41e6-8d74-8b50232428f2 not found for run 892a170d-b617-480b-86b6-701331f642d9. Treating as a root run.


{'answer': 'Yes, your name is Vinay.'}

In [72]:
store['abc123']

InMemoryChatMessageHistory(messages=[HumanMessage(content='My name is Vinay'), AIMessage(content="Nice to meet you, Vinay! I'm here to help with any questions or topics you'd like to discuss."), HumanMessage(content='Remember my Name?'), AIMessage(content='Yes, your name is Vinay.')])

In [19]:

from langchain_community.chat_models.ollama import ChatOllama
llm = ChatOllama(model="llama3", temperature=0)

In [20]:
llm.invoke("Tell me a joke")

AIMessage(content="Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(Wait for it...)\n\nBecause it was two-tired!\n\nHope that made you smile! Do you want to hear another one?", response_metadata={'model': 'llama3', 'created_at': '2024-07-13T21:01:15.426964Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'prompt_eval_count': 14, 'prompt_eval_duration': 92904000, 'eval_count': 41, 'eval_duration': 673141000}, id='run-dc19a906-becd-40f5-914c-c37f2d81d565-0')